### 1.2.1 BeautifulSoup
---
- BeautifulSoup 라이브러리에서 가장 널리 쓰이는 객체는 물론 BeautifulSoup 객체이다.

In [1]:
from urllib.request import urlopen
from bs4 import BeautifulSoup

html = urlopen('http://www.pythonscraping.com/pages/page1.html')

# HTML 콘텐츠 얻기
bs = BeautifulSoup(html.read(), 'html.parser')
print(bs.h1)

ModuleNotFoundError: No module named 'bs4'

- 객체를 만들 때는 두 가지 매개변수가 들어간다.
- `bs = BeautifulSoup(html.read(), 'html.paser')`
- `html.read` 는 이 객체의 근간이 되는 HMTL 텍스트이고,
- `html.parser` 는 BeautifulSoup가 객체를 만들 때 쓰는 구문 분석기인데, 이 구문 분석기는 우리가 직접 지정할 수 있다
  - `html.parser` 는 파이썬3와 함께 설치되므로 따로 설치할 필요는 없다.
  - lxml(http://lxml.de/parsing.html)도 널리 쓰이는 구문 분석기이다.
    - `pip3 install lxml`
    - `bs = BeautifulSoup(html.read(), 'lxml')`
    - lxml은 형식을 정확히 지키지 않은 '지저분한' HTML 코드를 분석할 때 html.parser보다 좀 더 낫다.
    - lxml은 닫히지 않은 태그, 계층 구조가 잘못된 태그, <head>나 <body> 태그가 없는 등의 문제에서 일일히 멈추지 않고 그 문제를 수정한다.
    - lxml은 html.parser에 비해 조금 더 빠르기도 하지만, 웹 스크래핑 분야에서는 네트워크 속도가 가장 큰 병목일 때가 대부분이므로 속도가 빠른 것은 별다른 장점이라 하기 어렵다.
  - html5lib도 널리 쓰이는 HTMl 구문 분석기이다.
    - lxml과 마찬가지로 html5lib도 잘못 만들어진 HTML을 수정하고 구문 분석을 시도하는데, lxml보다 더 다양한 에러를 수정할 수 있다.
    - 외부 프로그램이 있어야 동작할 수 있는 것도 마찬가지이며 lxml이나 html.parser보다 조금 느리다는 단점이 있긴 하지만, 잘못 만들어진 HTML이나 손으로 쓴 HTML을 분석해야 한다면 html5lib도 좋은 선택이다.